#Fingerprint+MD

### [데이터세트 구축]

#0801 FTO 학습 및 테스트 시작
* potency 1 (235건) potency 0 (118건) 총 535건 
* Potency 0은 실험데이터이므로 전부 들어가도록하고, 모자라는 부분만 decoy 랜덤으로 select 해서 총 LABEL0:LABEL1 = 1: 1,3,5,10,20,50 데이터 세트 구축함. 

#### 1. CPI 데이터 수집

1. ChemBL, Binding DB (Alpha-ketoglutarate-dependent dioxygenase FTO, Uniprot ID Q9C0B1) 에서 CPI 데이터 다운로드 후 DB구축 -> 1차 중복제거
2. Canonical smiles로 변환 후 2차 중복제거
3. 결과; potency 1 (235건) potency 0 (118건) 총 535건 

In [ ]:
#Canonical smiles변환

#필요 패키지 불러오기
from rdkit import Chem
import pandas as pd

#데이터 불러오기
dataset = pd.read_csv(r"C:\FTO_data\FTO_data.csv")
print("Data shape:", dataset.shape)
print(dataset.head(), "\n")

In [ ]:

#원본 SMILES 컬럼명 바꾸기
dataset = dataset.rename(columns={'Ligand SMILES': 'raw_SMILES'})
print("컬럼명 변경 후:", dataset.columns.tolist(), "\n")

#Canonical smiles 생성 
def canonical_smiles(smiles_list, include_stereo=True): #include_stereo: True면 입체화학(@,@@,/,\\) 정보 보존
    canonical = [] #결과를 담을 빈 리스트
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        #Mol이 None이 아니면 Moltosmiles, 아니면 None
        canonical.append(Chem.MolToSmiles(mol, isomericSmiles=include_stereo) if mol else None)
    return canonical

#canonical_SMILES컬럼을 추가하여 데이터 저장
Canon_SMILES = canonical_smiles(dataset['raw_SMILES']) #리스트 생성
assert len(Canon_SMILES) == len(dataset), "길이가 맞지 않습니다!" #원본 행 개수와 동일한지 검증 
dataset.insert(0, 'canonical_SMILES', Canon_SMILES)
print("데이터 shape (행, 열):", dataset.shape)
print(dataset[['canonical_SMILES', 'raw_SMILES']].head())

In [ ]:

#Create a list for duplicate smiles
duplicates_smiles = dataset[dataset['canonical_SMILES'].duplicated()]['canonical_SMILES'].values
print("제거될 중복 개수:", len(duplicates_smiles))

# 중복된 행 확인 (정렬)
dup_rows = dataset[dataset['canonical_SMILES'].isin(duplicates_smiles)].sort_values(by=['canonical_SMILES'])
print("중복된 샘플들:")
print(dup_rows[['canonical_SMILES', 'raw_SMILES']].head(), "\n")

#Drop duplicate values
dataset_unique = dataset.drop_duplicates(subset=['canonical_SMILES'])
print("중복 제거 후 남은 행 수:", len(dataset_unique))
print(dataset_unique.head())

dataset_unique.to_csv(r"C:\FTO_data\FTO_data_cannon.csv", index=False)

#### 2. DUDE 데이터 수집

1. DUDE생성 https://dude.docking.org/ 이용. (생성에 오류떠서 안씀; https://tldr.docking.org/)
2. 파일 합치기
3. DUDE cannon smiles로 변경 -> 중복제거 (코드) -> 중복제거 1회 더함 (1개 제거됨.) Decoy 15564건

In [ ]:
#decoy 생성을 위한 positive 뽑기

import pandas as pd
import numpy as np
import os
import random

#파일 경로 지정
n = 109 #필요한 샘플 갯수 지정
in_path = r"C:\path\to\input.csv"          # 원본 CSV
out_path = r"C:\path\to\positives_109.csv" # 결과를 다른 이름으로 저장

#input파일을 데이터프레임으로 저장
df = pd.read_csv(in_path)

# 컬럼 potency == 1인 행만 선택
label_1_data = df[df['potency'] == 1]

# 무작위로 n개 선택
random_selected_data = label_1_data.sample(n=n, random_state=42) #랜덤 시드

# 결과 저장
random_selected_data.to_csv(out_path, index=False)

In [ ]:
#DUDE 파일 병합

import pandas as pd
import os

# decoy 폴더 경로
folder_path = r"C:\FTO_data\decoy\output\0812\decoy_total"

# 모든 파일 경로 가져오기
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]

combined_df = pd.DataFrame()

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # 첫 번째 행 제거, 각 행을 tab으로 split, 2번째와 3번째 열 제거
    processed = []
    for line in lines[1:]:
        cols = line.strip().split('\t')
        # 인덱스 1과 2(2번째, 3번째 열) 제거
        filtered_cols = [col for idx, col in enumerate(cols) if idx not in [1, 2]]
        processed.append(filtered_cols)
    
    df = pd.DataFrame(processed)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# 결과 저장
combined_df.to_csv(r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy.csv", index=False)

In [ ]:
#Canonical smiles변환

#필요 패키지 불러오기
from rdkit import Chem
import pandas as pd

#데이터 불러오기
dataset = pd.read_csv(r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy.csv")
print("Data shape:", dataset.shape)
print(dataset.head(), "\n")

#원본 SMILES 컬럼명 바꾸기
dataset = dataset.rename(columns={'0': 'raw_SMILES'})
print("컬럼명 변경 후:", dataset.columns.tolist(), "\n")

#Canonical smiles 생성 
def canonical_smiles(smiles_list, include_stereo=True): #include_stereo: True면 입체화학(@,@@,/,\\) 정보 보존
    canonical = [] #결과를 담을 빈 리스트
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        #Mol이 None이 아니면 Moltosmiles, 아니면 None
        canonical.append(Chem.MolToSmiles(mol, isomericSmiles=include_stereo) if mol else None)
    return canonical

#canonical_SMILES컬럼을 추가하여 데이터 저장
Canon_SMILES = canonical_smiles(dataset['raw_SMILES']) #리스트 생성
assert len(Canon_SMILES) == len(dataset), "길이가 맞지 않습니다!" #원본 행 개수와 동일한지 검증 
dataset.insert(0, 'canonical_SMILES', Canon_SMILES)
print("데이터 shape (행, 열):", dataset.shape)
print(dataset[['canonical_SMILES', 'raw_SMILES']].head())

#Create a list for duplicate smiles
duplicates_smiles = dataset[dataset['canonical_SMILES'].duplicated()]['canonical_SMILES'].values
print("제거될 중복 개수:", len(duplicates_smiles))

# 중복된 행 확인 (정렬)
dup_rows = dataset[dataset['canonical_SMILES'].isin(duplicates_smiles)].sort_values(by=['canonical_SMILES'])
print("중복된 샘플들:")
print(dup_rows[['canonical_SMILES', 'raw_SMILES']].head(), "\n")

#Drop duplicate values
dataset_unique = dataset.drop_duplicates(subset=['canonical_SMILES'])
print("중복 제거 후 남은 행 수:", len(dataset_unique))
print(dataset_unique.head())

dataset_unique.to_csv(r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy_cannon.csv", index=False)

#### 3. 최종 데이터 세트 생성

In [ ]:
#비율 조절
import pandas as pd
import os


# ----------------- 데이터 불러오기 -----------------

# 파일 경로 설정
file1 = r"C:\FTO_data\FTO_data_cannon.csv"          # 실험 데이터 (potency=1,0 등)
file2 = r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy_cannon.csv"      # decoy 데이터셋 (potency=0)
outpath = r"C:\FTO_data"

#데이터 생성 비율 설정
ratios = [1, 3, 5, 10, 20, 50]

# CSV 파일 읽기
df1 = pd.read_csv(file1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
df2 = pd.read_csv(file2)

print("FTO_data_cannon.csv:", df1.shape)
print("FTO_DUDEdata_cannon.csv:", df2.shape)

# 열 구조 동일한지 확인
assert set(df1.columns) == set(df2.columns), "두 파일의 컬럼 구조가 다릅니다!"


In [ ]:
# ----------------- 실측 active/inactive 분리 -----------------
act = df1[df1["potency"] == 1].copy()
ina = df1[df1["potency"] == 0].copy()

POS = len(act)       # ex) 235
NEG = len(ina)   # ex) 118

In [ ]:
# ----------------- decoy 풀 준비 (모두 0 라벨로 가정) -----------------
sm_col = "canonical_SMILES" 

dec = df2.copy()
if "potency" not in dec.columns:
    dec["potency"] = 0
else:
    dec = dec[dec["potency"] == 0].copy()

# 실측과 decoy 간 SMILES 중복 제거(실측에 있는 건 decoy에서 제거)
dec_before = len(dec)
dec = dec[~dec[sm_col].isin(pd.concat([act[sm_col], ina[sm_col]]))]
print(f"[INFO] decoy duplicates removed: {dec_before - len(dec)}  (remain {len(dec)})")


In [ ]:
# 선택적으로 'source' 컬럼 추가(분석 편의)
act["source"] = "active"
ina["source"] = "assay_inactive"
dec["source"] = "decoy"

In [ ]:
# ----------------- 출력 폴더 -----------------
out_dir = os.path.join(outpath, "FTO_training")
os.makedirs(out_dir, exist_ok=True)

summary = []  # 요약 정보 저장용

# ----------------- 비율별 생성 -----------------

RANDOM_STATE = 42

# ----------------- 비율별 생성 -----------------
for r in ratios:
    target_neg = r * POS                         # 목표 음성 개수(= inactive+decoy)
    need_decoy = max(0, target_neg - NEG)    # 부족분은 decoy로 채움

    if need_decoy > len(dec):
        print(f"[WARN] ratio {r}: decoy 부족 -> {need_decoy}개 필요, {len(dec)}개만 사용")
        need_decoy = len(dec)
        
    # decoy 샘플링 (need_decoy가 0이면 빈 DF)
    dec_sample = dec.sample(n=need_decoy, random_state=RANDOM_STATE) if need_decoy > 0 else dec.iloc[0:0]

    # 합치기: active 전부 + inactive 전부 + decoy 일부
    df_ratio = pd.concat([act, ina, dec_sample], ignore_index=True)

    # 저장
    out_file = os.path.join(out_dir, f"FTO_training_{r}x.csv")
    df_ratio.to_csv(out_file, index=False, encoding="utf-8-sig")

    # 로그 & 요약 수집
    print(f"[SAVE] {out_file} -> pos={POS}, neg(real)={NEG}, decoy={len(dec_sample)}, total={len(df_ratio)}")
    summary.append({
        "ratio_neg_per_pos": f"1:{r}",
        "pos": POS,
        "neg_real": NEG,
        "decoy_used": len(dec_sample),
        "total": len(df_ratio),
        "path": out_file
    })

# ----------------- 요약표 출력 -----------------
summary_df = pd.DataFrame(summary)
print("\n=== Dataset summary by ratio ===")
print(summary_df)

#### 4. Fingerprint생성

In [ ]:
## 라이브러리 임포트 ##
import os                         # 파일 경로 및 디렉토리 작업을 위한 표준 라이브러리
import re                         # 정규표현식 처리 (문자열 패턴 검색 등)
import glob                       # 파일 경로에서 와일드카드 사용 가능하게 하는 모듈 (ex. *.csv 찾기)
import pickle                     # 파이썬 객체 저장 및 불러오기 (모델 저장/로드)
import pandas as pd               # 데이터프레임 생성 및 CSV/Excel 입출력 등 데이터 처리
import numpy as np                # 수치 계산 및 배열 처리 (벡터/행렬 등)
import multiprocessing            # 병렬 처리 (코어 여러 개 사용)
from collections import Counter   # 리스트 등에서 항목 개수 세기 (빈도 계산)

# 병렬처리 관련 라이브러리들
from concurrent.futures import ProcessPoolExecutor, as_completed  # 고수준 병렬처리 API
from joblib import Parallel, delayed                               # sklearn과 사용하기 좋은 병렬처리 도구
from tqdm import tqdm               # 반복문 진행 상황 시각화 (진행바 출력)

# RDKit: 화합물 처리 및 fingerprint 계산
from rdkit import Chem, DataStructs                      # SMILES → Mol, Mol 간 유사도 계산 등
from rdkit.Chem import AllChem, MACCSkeys, RDKFingerprint  # ECFP4, MACCS, RDKit FP 계산

# Mordred: 분자 descriptor 계산 라이브러리
from mordred import Calculator, descriptors              #Molecular descriptor 계산

In [ ]:
##Fingerprint 생성 함수 (ECFP4)##

# 개별 SMILES → ECFP4 fingerprint (bit-vector 리스트)
def smiles_to_fp(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
    arr = np.zeros((nBits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# 병렬처리 포함 전처리 함수
def process_smiles_fingerprints(smiles_list, radius=2, nBits=1024, workers=None, show_progress=True):
    # 병렬 작업에 사용할 워커 수 설정 (사용 가능한 CPU 코어 수-4)
    if workers is None:
        total_cores = multiprocessing.cpu_count()
        workers = max(1, total_cores - 4)

    # 결측값(NaN) 제거: SMILES 값이 있는 것만 필터링
    smiles_list_cleaned = [s for s in smiles_list if pd.notnull(s)]

    # tqdm을 사용해 진행률 표시 여부 설정
    iterator = tqdm(smiles_list_cleaned) if show_progress else smiles_list_cleaned

    # 병렬 처리로 SMILES → fingerprint 변환 수행
    fingerprints = Parallel(n_jobs=workers)(delayed(smiles_to_fp)(smi) for smi in iterator)

    # 변환 결과 중 None이 아닌 유효한 SMILES만 추출
    valid_data = [(smi, fp) for smi, fp in zip(smiles_list_cleaned, fingerprints) if fp is not None]

    # 유효한 결과가 하나도 없을 경우 빈 DataFrame 반환
    if not valid_data:
        print("유효한 fingerprint가 없습니다.")
        return pd.DataFrame()

    # 유효한 SMILES와 해당 fingerprint 배열 분리
    valid_smiles, valid_fps = zip(*valid_data)

    # fingerprint의 컬럼명을  1x ~ nBits x 형식으로 지정
    bit_columns = [f"X{i+1}" for i in range(nBits)]
    
    # 결과를 DataFrame으로 구성
    fp_df = pd.DataFrame(valid_fps, columns=bit_columns)
    fp_df.insert(0, 'SMILES', valid_smiles)  # 첫 번째 열에 SMILES 추가

    # 결과 요약 출력
    print(f"총 {len(smiles_list)}개의 SMILES 중 {len(valid_smiles)}개가 정상적으로 fingerprint로 변환되었습니다.")

    return fp_df

In [ ]:
## 호출부

folder_path = r"C:\FTO_data\decoy\output\0812\decoy_total"

# 반복 처리
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)

    # CSV 불러오기
    df = pd.read_csv(file_path)

    # 유효한 SMILES만 추출
    smiles_col = 'canonical_SMILES'
    df_clean = df[pd.notnull(df[smiles_col])].reset_index(drop=True)
    smiles_list = df_clean[smiles_col].tolist()

    # Fingerprint 생성
    fingerprint_df = process_smiles_fingerprints(smiles_list)

    # SMILES 컬럼 제거 후 원본 메타데이터와 fingerprint 결합
    fingerprint_only = fingerprint_df.drop(columns=['SMILES'])  # 중복 방지

    # 최종 결합 (원본 df_clean + fingerprint)
    final_df = pd.concat([df_clean, fingerprint_only], axis=1)

    # 결과 shape 출력
    print(f"📁 처리된 파일: {file_name}")
    print("📊 최종 결과 shape:", final_df.shape)

    # 결과 저장
    output_file_name = os.path.splitext(file_name)[0] + "_ecfp4.csv"
    output_path = os.path.join(folder_path, output_file_name)
    final_df.to_csv(output_path, index=False)
    print(f"✅ 저장 완료: {output_path}\n")

### [모델학습]

#### 1. 데이터 불러오기 (ECFP4 fingerprint)

In [7]:
## 라이브러리 임포트 ##
from sklearn.model_selection import train_test_split 
import logging
import os
from datetime import datetime
import pandas as pd
import re

In [19]:
##환경설정##

#로깅 설정
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s] %(name)s: %(message)s')

#경로지정
INPATH=r"C:\FTO_data\FTO_training\data_20250828\md_igonore3D_T"
today = datetime.today().strftime('%Y%m%d') 
OUTPATH = os.path.join(r"C:\FTO_data\FTO_training", f"data_{today}")
if not os.path.exists(OUTPATH):
    os.makedirs(OUTPATH)

In [20]:
# 처리할 파일 목록
file_list = [f for f in os.listdir(INPATH) if f.lower().endswith('.csv')]

# 정규식: 접미사만 추출할 때 사용할 패턴
suffix_pattern = re.compile(r'_(\d+x)')

# 최종 데이터들을 저장할 딕셔너리
train_dfs = {} # 모델링에 사용할 데이터 (입력 + 타겟)
metadata_dfs = {} # 메타데이터

for file_name in file_list:
    file_path = os.path.join(INPATH, file_name)
    logging.info(f"🔍 처리 중: {file_name}")

    try:
        # 1. 파일 불러오기
        df = pd.read_csv(file_path)
        
        # 2. 메타데이터로 사용할 열 이름 정의
        meta_cols = ['canonical_SMILES', 'raw_SMILES', 'source']
        
        # 3. 데이터 분리
        # 메타데이터 추출
        df_meta = df[meta_cols].copy()
        
        # 모델링용 데이터 추출: 전체 열에서 메타데이터 열만 제외
        train_cols = df.columns.drop(meta_cols)
        df_train = df[train_cols].copy()
    
        # 4. 결과 저장
        # 파일 이름에서 접미사(1x 등) 추출
        match = suffix_pattern.search(file_name)
        suffix = match.group(1) if match else 'unknown'

        # 각 딕셔너리에 접미사를 키로 하여 데이터프레임 저장
        train_dfs[f'df_train_{suffix}'] = df_train
        metadata_dfs[f'metadata_{suffix}'] = df_meta

        logging.info(f"✅ 데이터 분리 완료: {file_name}")
        logging.info(f"  -> df_train_{suffix}: {df_train.shape}")
        logging.info(f"  -> metadata_{suffix}: {df_meta.shape}")

        # globals()를 사용해 직접 변수 생성
        globals()[f'df_train_{suffix}'] = df_train
        globals()[f'metadata_{suffix}'] = df_meta

    except KeyError as e:
        logging.error(f"❌ 처리 실패: {file_name} - 열 이름 확인 필요: {e}")
    except Exception as e:
        logging.error(f"❌ 처리 실패: {file_name} - {str(e)}")

2025-08-28 00:37:43,063 [INFO] root: 🔍 처리 중: filtered_FTO_training_10x_ignore3D_True.csv
2025-08-28 00:37:43,609 [INFO] root: ✅ 데이터 분리 완료: filtered_FTO_training_10x_ignore3D_True.csv
2025-08-28 00:37:43,610 [INFO] root:   -> df_train_10x: (2585, 1367)
2025-08-28 00:37:43,611 [INFO] root:   -> metadata_10x: (2585, 3)
2025-08-28 00:37:43,615 [INFO] root: 🔍 처리 중: filtered_FTO_training_20x_ignore3D_True.csv
2025-08-28 00:37:44,578 [INFO] root: ✅ 데이터 분리 완료: filtered_FTO_training_20x_ignore3D_True.csv
2025-08-28 00:37:44,580 [INFO] root:   -> df_train_20x: (4935, 1367)
2025-08-28 00:37:44,580 [INFO] root:   -> metadata_20x: (4935, 3)
2025-08-28 00:37:44,586 [INFO] root: 🔍 처리 중: filtered_FTO_training_5x_ignore3D_True.csv
2025-08-28 00:37:44,903 [INFO] root: ✅ 데이터 분리 완료: filtered_FTO_training_5x_ignore3D_True.csv
2025-08-28 00:37:44,903 [INFO] root:   -> df_train_5x: (1410, 1367)
2025-08-28 00:37:44,903 [INFO] root:   -> metadata_5x: (1410, 3)


In [21]:
# 분리 대상 접미사
data_suffixes = ['5x', '10x', '20x']

for suffix in data_suffixes:
    try:
        # 1. 이전 단계에서 만든 df_train_{suffix} 변수를 불러옵니다.
        df_full = globals()[f'df_train_{suffix}']

        # 2. 불러온 데이터프레임에서 X와 Y를 분리합니다.
        y_col = 'potency'
        x_cols = df_full.columns.drop(y_col)
        
        x_full = df_full[x_cols]
        y_full = df_full[y_col]

        # 3. train/test 분리를 수행합니다.
        x_train, x_test, y_train, y_test = train_test_split(
            x_full, y_full,
            test_size=0.1,
            random_state=42,
            stratify=y_full
        )

        # 4. 분리된 결과를 새로운 변수로 저장합니다.
        globals()[f'x_train_{suffix}'] = x_train
        globals()[f'y_train_{suffix}'] = y_train
        globals()[f'x_test_{suffix}'] = x_test
        globals()[f'y_test_{suffix}'] = y_test
        
        logging.info(
            f"✅ {suffix} 데이터 → train/test 분리 완료: "
            f"x_train{x_train.shape}, y_train{y_train.shape}, "
            f"x_test{x_test.shape}, y_test{y_test.shape}"
        )
        
    except KeyError:
        logging.error(f"❌ 변수 로드 실패: df_train_{suffix}를 찾을 수 없습니다.")
    except Exception as e:
        logging.error(f"❌ {suffix} 데이터 분리 실패: {str(e)}")

2025-08-28 00:37:53,357 [INFO] root: ✅ 5x 데이터 → train/test 분리 완료: x_train(1269, 1366), y_train(1269,), x_test(141, 1366), y_test(141,)
2025-08-28 00:37:53,402 [INFO] root: ✅ 10x 데이터 → train/test 분리 완료: x_train(2326, 1366), y_train(2326,), x_test(259, 1366), y_test(259,)
2025-08-28 00:37:53,482 [INFO] root: ✅ 20x 데이터 → train/test 분리 완료: x_train(4441, 1366), y_train(4441,), x_test(494, 1366), y_test(494,)


In [22]:
x_train_5x.shape

(1269, 1366)

#### 2. 모델학습 (PyCaret)

In [23]:
from pycaret.classification import *

2025-08-28 00:38:12,184 [DEBUG] matplotlib: matplotlib data path: c:\Users\user\anaconda3\envs\my-rdkit-env\lib\site-packages\matplotlib\mpl-data
2025-08-28 00:38:12,186 [DEBUG] matplotlib: CONFIGDIR=C:\Users\user\.matplotlib
2025-08-28 00:38:12,196 [DEBUG] matplotlib: interactive is False
2025-08-28 00:38:12,196 [DEBUG] matplotlib: platform is win32
2025-08-28 00:38:12,287 [DEBUG] matplotlib: CACHEDIR=C:\Users\user\.matplotlib
2025-08-28 00:38:12,297 [DEBUG] matplotlib.font_manager: Using fontManager instance from C:\Users\user\.matplotlib\fontlist-v330.json
2025-08-28 00:38:14,092 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:38:14,092 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [24]:
#데이터 전처리 및 모델학습

# 모델 비교 결과를 데이터프레임으로 저장할 딕셔너리
all_comparison_df = {} 
data_suffixes = ['5x', '10x', '20x']

for suffix in data_suffixes:
    x_train = globals()[f'x_train_{suffix}']
    y_train = globals()[f'y_train_{suffix}']
    df_train = pd.concat([x_train, y_train], axis=1)

    # 데이터 셋업
    setup(
        data=df_train,
        target='potency',
        session_id=42,
        train_size=0.9,
        fold=10, #디폴트
        normalize=True,
        fix_imbalance=True,
        remove_outliers=True,
        n_jobs=-1, 
        verbose=False
    )

    print(f"✅ [{suffix}] 모델 비교 시작")    
    
    # 모델 비교
    compare_models(fold=5, sort='F1', n_select=15, turbo=True)
    
    # [핵심] 결과를 pull()을 이용해 데이터프레임으로 저장합니다.
    results_df = pull()
    all_comparison_df[suffix] = results_df
    
    print(f"✅ [{suffix}] 결과 저장 완료")

print("\n모든 데이터세트의 모델 비교 결과(데이터프레임) 저장이 완료되었습니다!")

2025-08-28 00:38:30,216 [DEBUG] graphviz._tools: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2025-08-28 00:38:30,219 [DEBUG] graphviz._tools: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2025-08-28 00:38:30,221 [DEBUG] graphviz._tools: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2025-08-28 00:38:30,224 [DEBUG] graphviz._tools: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2025-08-28 00:38:30,226 [DEBUG] graphviz._tools: deprecate positional args: graphviz.quoting.quote(['is_html_string', 'is_valid_id', 'dot_keywords', 'endswith_odd_number_of_backslashes', 'escape_unescaped_quotes'])
2025-08-28 00:38:30,231 [DEBUG] graphviz._tools: deprecate positional args: graphviz.quoting.a_list(['kwargs', 'attributes'])
2025-08-28 00:38:30,231 [DEBUG] graphviz._tools: deprecate positi

✅ [5x] 모델 비교 시작


2025-08-28 00:38:44,266 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


2025-08-28 00:38:44,275 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9457,0.9713,0.8062,0.8636,0.8326,0.8003,0.8018,5.0640
lr,Logistic Regression,0.9448,0.9713,0.8009,0.8619,0.8294,0.7966,0.7979,1.6660
et,Extra Trees Classifier,0.9466,0.9686,0.7748,0.8937,0.8290,0.7976,0.8011,0.7280
lightgbm,Light Gradient Boosting Machine,0.9440,0.9708,0.7748,0.8782,0.8227,0.7896,0.7921,12.4700
rf,Random Forest Classifier,0.9405,0.9755,0.7381,0.8871,0.8052,0.7705,0.7753,0.8660
svm,SVM - Linear Kernel,0.9387,0.9654,0.7229,0.8954,0.7975,0.7620,0.7696,0.7540
ada,Ada Boost Classifier,0.9308,0.9648,0.7644,0.8149,0.7877,0.7465,0.7478,1.5040
dt,Decision Tree Classifier,0.9072,0.8544,0.7750,0.7098,0.7374,0.6814,0.6849,0.7800
nb,Naive Bayes,0.8905,0.7209,0.4660,0.7992,0.5876,0.5295,0.5557,1.3940
ridge,Ridge Classifier,0.7960,0.8714,0.8008,0.4394,0.5674,0.4483,0.4831,0.8440


...

2025-08-28 00:48:30,692 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9827,0.9592,0.8158,0.8220,0.8177,0.8086,0.8093,0.8820
lightgbm,Light Gradient Boosting Machine,0.9825,0.9740,0.7474,0.8639,0.8004,0.7913,0.7941,8.2920
et,Extra Trees Classifier,0.9825,0.9840,0.7316,0.8791,0.7975,0.7884,0.7926,1.1140
gbc,Gradient Boosting Classifier,0.9807,0.9703,0.7579,0.8224,0.7877,0.7777,0.7790,17.1840
ridge,Ridge Classifier,0.9767,0.9478,0.8526,0.7157,0.7772,0.7650,0.7688,1.1020
rf,Random Forest Classifier,0.9805,0.9818,0.6895,0.8712,0.7687,0.7587,0.7649,1.2760
lda,Linear Discriminant Analysis,0.9745,0.9478,0.8474,0.6900,0.7599,0.7466,0.7514,1.8320
svm,SVM - Linear Kernel,0.9790,0.9670,0.6579,0.8698,0.7480,0.7373,0.7457,1.0560
ada,Ada Boost Classifier,0.9762,0.9516,0.7316,0.7628,0.7456,0.7331,0.7340,5.0560
dt,Decision Tree Classifier,0.9652,0.8567,0.7368,0.6143,0.6687,0.6506,0.6543,1.8280


2025-08-28 00:48:30,692 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


2025-08-28 00:52:44,623 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:52:44,624 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:52:50,020 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:52:50,024 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:53:00,838 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:53:00,841 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:53:03,905 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:53:03,907 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


✅ [20x] 결과 저장 완료

모든 데이터세트의 모델 비교 결과(데이터프레임) 저장이 완료되었습니다!


In [25]:
##상위 모델 추출
top_models_set = set()

# 저장된 결과 데이터프레임에서 직접 모델 ID를 추출합니다.
for suffix, results_df in all_comparison_df.items():
    
    # [핵심] 데이터프레임의 인덱스가 바로 모델 ID입니다. 상위 5개를 가져옵니다.
    top_5_ids = results_df.index[:5].tolist()
    
    # .update()를 사용해 리스트의 모든 항목을 set에 추가
    top_models_set.update(top_5_ids)

final_tuning_list = list(top_models_set)

print("✅ 튜닝 대상 모델 목록이 생성되었습니다.")
print(f" -> 최종 튜닝 모델: {final_tuning_list}")

✅ 튜닝 대상 모델 목록이 생성되었습니다.
 -> 최종 튜닝 모델: ['ada', 'ridge', 'lr', 'et', 'rf', 'lightgbm', 'gbc']


In [26]:
##하이퍼파라미터 튜닝

# 결과를 저장할 최종 딕셔너리 (사용자 요청 형식)
all_model_dfs = {}    # 모델별 전체 fold 성능 저장
all_summary_dfs = {}  # 데이터세트별로 각 모델의 성능 요약
all_tuned_models = {}   # 튜닝된 모델 객체 저장

data_suffixes = ['5x', '10x', '20x']

print("=============== 하이퍼파라미터 튜닝 및 최종 결과 저장 시작 ===============")

for suffix in data_suffixes:
    print(f"\033[1;30m\n------------ 데이터세트 [{suffix}] 튜닝 시작 ------------\033[0m")
    
    # 데이터 준비
    x_train = globals()[f'x_train_{suffix}']
    y_train = globals()[f'y_train_{suffix}']
    df_train = pd.concat([x_train, y_train], axis=1)
    
    # PyCaret 설정
    data_setup = setup(
        data=df_train,
        target='potency',
        session_id=42,
        train_size=0.9,
        fold=10, #디폴트
        normalize=True,
        fix_imbalance=True,
        remove_outliers=True,
        n_jobs=-1,
        verbose=False
    )
    
    # 현재 suffix의 결과를 임시 저장할 변수들
    model_dfs = {}
    tuned_models = {} 
    summary_rows = []
    
    # 모델 튜닝
    for model_id in final_tuning_list:
        try:
            print(f"\033[1;31m\n  -> 모델 [{model_id}] 튜닝 중...\033[0m")
            
            # 튜닝할 모델 생성
            model_to_tune = create_model(model_id, verbose=False)
            
            # 모델 튜닝
            tuned_model = tune_model(
                model_to_tune,
                optimize='F1', 
                n_iter=100,
                verbose=True)
            
            result_grid = pull().copy()
            
            # 모델 이름 추출 (예: 'ExtraTreesClassifier')
            model_name = str(tuned_model).split('(')[0]
            
            # Mean, Std 행을 나중에 식별하기 위해 'Type' 열 추가
            result_grid['Model'] = model_name
            result_grid['Type'] = result_grid.index
            
            # 각 딕셔너리에 결과 저장
            model_dfs[model_name] = result_grid
            tuned_models[model_name] = tuned_model
            summary_rows.append(result_grid[result_grid['Type'].isin(['Mean', 'Std'])])
            
            print(f"  ✅ 튜닝 완료!")
            
        except Exception as e:
            logging.error(f"  ❌ 모델 [{model_id}] 튜닝 실패: {e}")
            
    # 현재 suffix의 요약(summary) 데이터프레임 생성
    summary_df = pd.concat(summary_rows, ignore_index=True)
    
    # 최종 딕셔너리에 현재 suffix의 결과들을 저장
    all_model_dfs[suffix] = model_dfs
    all_summary_dfs[suffix] = summary_df
    all_tuned_models[suffix] = tuned_models

    print(f"\n[완료] {suffix} 튜닝 및 저장 ✅")
    
#모델 및 평가지표 저장
print("\n🎉 모든 튜닝 및 저장 작업이 완료되었습니다!")

=============== 하이퍼파라미터 튜닝 및 최종 결과 저장 시작 ===============

------------ 데이터세트 [5x] 튜닝 시작 ------------


2025-08-28 00:55:30,965 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 00:55:30,965 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.



  -> 모델 [ada] 튜닝 중...


2025-08-28 00:55:45,175 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


2025-08-28 00:55:45,181 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9925,0.9946,0.8947,0.9444,0.9189,0.9150,0.9154
1,0.9925,0.9985,0.8947,0.9444,0.9189,0.9150,0.9154
2,0.9825,0.9794,0.6842,0.9286,0.7879,0.7790,0.7889
3,0.9875,0.9967,0.8947,0.8500,0.8718,0.8652,0.8655
4,0.9800,0.9923,0.8421,0.7619,0.8000,0.7895,0.7906
5,0.9825,0.9431,0.6842,0.9286,0.7879,0.7790,0.7889
6,0.9799,0.9939,0.7895,0.7895,0.7895,0.7789,0.7789
7,0.9774,0.9507,0.6842,0.8125,0.7429,0.7312,0.7341
8,0.9799,0.9353,0.7368,0.8235,0.7778,0.7673,0.7686


2025-08-28 21:22:41,578 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


2025-08-28 23:18:58,755 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-28 23:18:58,765 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


  ✅ 튜닝 완료!

[완료] 20x 튜닝 및 저장 ✅

🎉 모든 튜닝 및 저장 작업이 완료되었습니다!


In [27]:
## 튜닝 결과비교

metric = "F1"  # 정렬 기준 (원하면 "AUC", "Accuracy" 등으로 변경)
cols = ["Model", "Accuracy", "AUC", "Recall", "Prec.", "F1", "Kappa", "MCC"]  # 보기용 컬럼

for suffix in data_suffixes:
    print(f"\033[1;30m\n[{suffix}] 결과\033[0m")

    # 1) 튜닝 전: 비교 결과 상위 5개
    print("하이퍼파라미터 튜닝 전 (Top 5)")
    results_df = all_comparison_df[suffix]
    # 정렬 기준 컬럼이 없을 수도 있으므로 방어
    if metric in results_df.columns:
        pre_top5 = results_df.sort_values(by=metric, ascending=False).head(5)
    else:
        pre_top5 = results_df

    display(pre_top5[cols] if set(cols).issubset(pre_top5.columns) else pre_top5)

    # 2) 튜닝 후: Mean 행만 필터 → 상위 5개
    print("하이퍼파라미터 튜닝 후")
    summary_df = all_summary_dfs[suffix]
    post_df = summary_df[summary_df["Type"] == "Mean"].copy()

    if post_df.empty:
        print(" - [알림] 'Mean' 행이 없습니다. summary_df 전체를 표시합니다.")
        display(summary_df)
        continue

    if metric in post_df.columns:
        post_top5 = post_df.sort_values(by=metric, ascending=False) #.head(6)
    else:
        post_top5 = post_df

    display(post_top5[cols] if set(cols).issubset(post_top5.columns) else post_top5)


[5x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
gbc,Gradient Boosting Classifier,0.9457,0.9713,0.8062,0.8636,0.8326,0.8003,0.8018
lr,Logistic Regression,0.9448,0.9713,0.8009,0.8619,0.8294,0.7966,0.7979
et,Extra Trees Classifier,0.9466,0.9686,0.7748,0.8937,0.8290,0.7976,0.8011
lightgbm,Light Gradient Boosting Machine,0.9440,0.9708,0.7748,0.8782,0.8227,0.7896,0.7921
rf,Random Forest Classifier,0.9405,0.9755,0.7381,0.8871,0.8052,0.7705,0.7753


하이퍼파라미터 튜닝 후


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
10,LGBMClassifier,0.9544,0.9812,0.8216,0.8996,0.8569,0.8299,0.8324
12,GradientBoostingClassifier,0.9545,0.9737,0.8168,0.9055,0.8559,0.8290,0.8326
6,ExtraTreesClassifier,0.9518,0.9647,0.8426,0.8699,0.8527,0.8241,0.8265
4,LogisticRegression,0.9510,0.9758,0.8326,0.8740,0.8483,0.8193,0.8226
8,RandomForestClassifier,0.9448,0.9663,0.7900,0.8747,0.8266,0.7941,0.7979
0,AdaBoostClassifier,0.9405,0.9727,0.7903,0.8466,0.8135,0.7784,0.7815
2,RidgeClassifier,0.8205,0.8866,0.8011,0.4795,0.5981,0.4921,0.5207



[10x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
4,LogisticRegression,0.9713,0.9779,0.8368,0.8518,0.8411,0.8254,0.8273
6,ExtraTreesClassifier,0.9723,0.9682,0.7895,0.8921,0.8344,0.8195,0.8231
10,LGBMClassifier,0.9713,0.9710,0.7895,0.8857,0.8319,0.8164,0.8197
12,GradientBoostingClassifier,0.9699,0.9736,0.7895,0.8731,0.8244,0.8081,0.8120
0,AdaBoostClassifier,0.9666,0.9740,0.7789,0.8444,0.8062,0.7881,0.7912
8,RandomForestClassifier,0.9637,0.9643,0.7474,0.8461,0.7858,0.7663,0.7725
2,RidgeClassifier,0.9274,0.9431,0.8316,0.5776,0.6770,0.6380,0.6544



[20x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
lr,Logistic Regression,0.9827,0.9592,0.8158,0.8220,0.8177,0.8086,0.8093
lightgbm,Light Gradient Boosting Machine,0.9825,0.9740,0.7474,0.8639,0.8004,0.7913,0.7941
et,Extra Trees Classifier,0.9825,0.9840,0.7316,0.8791,0.7975,0.7884,0.7926
gbc,Gradient Boosting Classifier,0.9807,0.9703,0.7579,0.8224,0.7877,0.7777,0.7790
ridge,Ridge Classifier,0.9767,0.9478,0.8526,0.7157,0.7772,0.7650,0.7688


하이퍼파라미터 튜닝 후


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
10,LGBMClassifier,0.9847,0.9792,0.7842,0.8929,0.8296,0.8217,0.8265
4,LogisticRegression,0.9832,0.9718,0.8158,0.8366,0.8222,0.8135,0.8156
12,GradientBoostingClassifier,0.9832,0.9733,0.7684,0.8700,0.8105,0.8019,0.8065
6,ExtraTreesClassifier,0.9817,0.9740,0.7737,0.8373,0.8001,0.7906,0.7935
2,RidgeClassifier,0.9777,0.9567,0.8632,0.7366,0.7899,0.7784,0.7838
8,RandomForestClassifier,0.9792,0.9637,0.7632,0.7990,0.7760,0.7652,0.7679
0,AdaBoostClassifier,0.9787,0.9754,0.7737,0.7882,0.7756,0.7645,0.7674


In [28]:
## 튜닝 결과비교

metric = "F1"  # 정렬 기준

for suffix in data_suffixes:
    print(f"\033[1;30m\n[{suffix}] 결과\033[0m")

    # --- 1) 튜닝 전: 비교 결과 상위 5개 ---
    print("하이퍼파라미터 튜닝 전 (Top 5)")
    results_df = all_comparison_df[suffix]
    pre_top5 = results_df.sort_values(by=metric, ascending=False).head(5)
    
    # [수정] 'Model' 열을 인덱스로 만들어서 출력합니다.
    display(pre_top5.set_index('Model'))

    # --- 2) 튜닝 후: Mean 행만 필터 → 상위 5개 ---
    print("\n하이퍼파라미터 튜닝 후 (Top 5)")
    summary_df = all_summary_dfs[suffix]
    post_df = summary_df[summary_df["Type"] == "Mean"].copy()

    if post_df.empty:
        print(" - [알림] 'Mean' 행이 없습니다.")
        continue

    # [수정] 'Model' 열을 인덱스로 설정합니다.
    post_df.set_index('Model', inplace=True)
    
    post_top5 = post_df.sort_values(by=metric, ascending=False).head(5)
    display(post_top5)


[5x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
Model,,,,,,,,
Gradient Boosting Classifier,0.9457,0.9713,0.8062,0.8636,0.8326,0.8003,0.8018,5.064
Logistic Regression,0.9448,0.9713,0.8009,0.8619,0.8294,0.7966,0.7979,1.666
Extra Trees Classifier,0.9466,0.9686,0.7748,0.8937,0.8290,0.7976,0.8011,0.728
Light Gradient Boosting Machine,0.9440,0.9708,0.7748,0.8782,0.8227,0.7896,0.7921,12.470
Random Forest Classifier,0.9405,0.9755,0.7381,0.8871,0.8052,0.7705,0.7753,0.866



하이퍼파라미터 튜닝 후 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Type
Model,,,,,,,,
LGBMClassifier,0.9544,0.9812,0.8216,0.8996,0.8569,0.8299,0.8324,Mean
GradientBoostingClassifier,0.9545,0.9737,0.8168,0.9055,0.8559,0.8290,0.8326,Mean
ExtraTreesClassifier,0.9518,0.9647,0.8426,0.8699,0.8527,0.8241,0.8265,Mean
LogisticRegression,0.9510,0.9758,0.8326,0.8740,0.8483,0.8193,0.8226,Mean
RandomForestClassifier,0.9448,0.9663,0.7900,0.8747,0.8266,0.7941,0.7979,Mean



[10x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Type
Model,,,,,,,,
LogisticRegression,0.9713,0.9779,0.8368,0.8518,0.8411,0.8254,0.8273,Mean
ExtraTreesClassifier,0.9723,0.9682,0.7895,0.8921,0.8344,0.8195,0.8231,Mean
LGBMClassifier,0.9713,0.9710,0.7895,0.8857,0.8319,0.8164,0.8197,Mean
GradientBoostingClassifier,0.9699,0.9736,0.7895,0.8731,0.8244,0.8081,0.8120,Mean
AdaBoostClassifier,0.9666,0.9740,0.7789,0.8444,0.8062,0.7881,0.7912,Mean



[20x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
Model,,,,,,,,
Logistic Regression,0.9827,0.9592,0.8158,0.8220,0.8177,0.8086,0.8093,0.882
Light Gradient Boosting Machine,0.9825,0.9740,0.7474,0.8639,0.8004,0.7913,0.7941,8.292
Extra Trees Classifier,0.9825,0.9840,0.7316,0.8791,0.7975,0.7884,0.7926,1.114
Gradient Boosting Classifier,0.9807,0.9703,0.7579,0.8224,0.7877,0.7777,0.7790,17.184
Ridge Classifier,0.9767,0.9478,0.8526,0.7157,0.7772,0.7650,0.7688,1.102



하이퍼파라미터 튜닝 후 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Type
Model,,,,,,,,
LGBMClassifier,0.9847,0.9792,0.7842,0.8929,0.8296,0.8217,0.8265,Mean
LogisticRegression,0.9832,0.9718,0.8158,0.8366,0.8222,0.8135,0.8156,Mean
GradientBoostingClassifier,0.9832,0.9733,0.7684,0.8700,0.8105,0.8019,0.8065,Mean
ExtraTreesClassifier,0.9817,0.9740,0.7737,0.8373,0.8001,0.7906,0.7935,Mean
RidgeClassifier,0.9777,0.9567,0.8632,0.7366,0.7899,0.7784,0.7838,Mean


In [29]:
import joblib

In [30]:
#모델 및 평가지표 저장
# 결과를 저장할 기본 경로 설정 (이전 코드와 동일하게)
today = datetime.today().strftime('%Y%m%d')
OUTPATH = os.path.join(r"C:\FTO_data\FTO_training", f"data_{today}")
os.makedirs(OUTPATH, exist_ok=True)

print(f"결과를 다음 경로에 저장합니다: {OUTPATH}")

# Cell 3에서 생성된 최종 결과 딕셔너리들을 순회합니다.
for suffix in all_model_dfs.keys():
    
    # 1. 각 데이터세트별로 결과 폴더 생성
    suffix_path = os.path.join(OUTPATH, f"{suffix}_mdT") ##폴더명: fingerprint 데이터 사용함.
    os.makedirs(suffix_path, exist_ok=True)
    print(f"\n--- [{suffix}] 데이터세트 결과 저장 중 ---")

    # 2. 요약 결과(summary_df) 저장
    summary_df = all_summary_dfs[suffix]
    summary_path = os.path.join(suffix_path, f"{suffix}_summary_evaluation.csv")
    summary_df.to_csv(summary_path, index=False)
    print(f"  -> 요약 결과 저장 완료: {summary_path}")

    # 3. 모델별 상세 평가결과(evaluation.csv) 저장
    model_dfs = all_model_dfs[suffix]
    for model_name, df in model_dfs.items():
        eval_path = os.path.join(suffix_path, f"{suffix}_{model_name}_evaluation.csv")
        df.to_csv(eval_path, index=False)
    print(f"  -> 모델별 상세 결과 {len(model_dfs)}개 저장 완료.")

    # 4. 튜닝된 모델(.pkl) 저장
    tuned_models = all_tuned_models[suffix]
    model_save_path = os.path.join(suffix_path, "models")
    os.makedirs(model_save_path, exist_ok=True)
    
    for model_name, model_object in tuned_models.items():
        model_path = os.path.join(model_save_path, f"{suffix}_{model_name}_model.pkl")
        joblib.dump(model_object, model_path)
    print(f"  -> 튜닝된 모델 {len(tuned_models)}개 저장 완료.")

print("\n🎉 모든 결과 파일 저장이 완료되었습니다!")

결과를 다음 경로에 저장합니다: C:\FTO_data\FTO_training\data_20250828

--- [5x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250828\5x_mdT\5x_summary_evaluation.csv
  -> 모델별 상세 결과 7개 저장 완료.
  -> 튜닝된 모델 7개 저장 완료.

--- [10x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250828\10x_mdT\10x_summary_evaluation.csv
  -> 모델별 상세 결과 7개 저장 완료.
  -> 튜닝된 모델 7개 저장 완료.

--- [20x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250828\20x_mdT\20x_summary_evaluation.csv
  -> 모델별 상세 결과 7개 저장 완료.
  -> 튜닝된 모델 7개 저장 완료.

🎉 모든 결과 파일 저장이 완료되었습니다!


In [32]:
import shutil

In [ ]:
# 모델 Plot 생성 및 저장 (validation dataset)

print(f"Plot을 다음 기본 경로에 저장합니다: {OUTPATH}")

# PyCaret이 plot_type별로 저장하는 기본 파일 이름 정의
default_filenames = {
    'auc': 'AUC.png',
    'confusion_matrix': 'Confusion Matrix.png',
    'learning': 'Learning Curve.png',
    'feature': 'Feature Importance.png',
    'error': 'Prediction Error.png',
    'calibration': 'Calibration Curve.png'
}

data_suffixes = ['5x', '10x', '20x']

for suffix in data_suffixes:
    
    # Plot을 그리기 전에 해당 데이터로 setup을 다시 실행하여 세션을 활성화
    print(f"\n--- 데이터세트 [{suffix}]에 대한 PyCaret 세션 활성화 ---")
    x_train = globals()[f'x_train_{suffix}']
    y_train = globals()[f'y_train_{suffix}']
    df_train = pd.concat([x_train, y_train], axis=1)
    
    # 튜닝 때 사용했던 설정과 동일하게 setup을 실행
    setup(
        data=df_train,
        target='potency',
        session_id=42,
        train_size=0.9,
        fold=10,
        normalize=True,
        fix_imbalance=True,
        remove_outliers=True,
        n_jobs=-1, 
        verbose=False
    )
    
    # Plot 저장 폴더 생성
    plot_save_dir = os.path.join(OUTPATH, f"{suffix}_mdT", "plots")
    os.makedirs(plot_save_dir, exist_ok=True)
    print(f"--- [{suffix}] 데이터세트 Plot 저장 시작 ---")

    # 튜닝된 모델들을 하나씩 순회
    for model_name, model_object in all_tuned_models[suffix].items():
        print(f"  -> 모델 [{model_name}]의 Plot 생성...")
        
        # 정의된 plot 종류를 순회
        for plot_type, default_name in default_filenames.items():
            try:
                # 1. 기본 이름으로 저장
                plot_model(model_object, plot=plot_type, save=True, verbose=False)
                
                # 2. 원하는 이름과 위치로 이동
                default_file_path = default_name
                final_filename = f"{suffix}_{model_name}_{plot_type}.png"
                final_save_path = os.path.join(plot_save_dir, final_filename)
                
                if os.path.exists(default_file_path):
                    shutil.move(default_file_path, final_save_path)
                else:
                    logging.warning(f"    - 기본 파일 '{default_file_path}'을 찾을 수 없습니다.")

            except Exception as e:
                logging.error(f"     - [{plot_type}] Plot 생성 실패: {e}")

    print(f"  ✅ [{suffix}] 데이터세트의 Plot 저장이 완료되었습니다.")

print("\n🎉 모든 Plot 파일 저장이 완료되었습니다!")

Plot을 다음 기본 경로에 저장합니다: C:\FTO_data\FTO_training\data_20250828

--- 데이터세트 [5x]에 대한 PyCaret 세션 활성화 ---
--- [5x] 데이터세트 Plot 저장 시작 ---
  -> 모델 [AdaBoostClassifier]의 Plot 생성...


2025-08-28 23:36:41,817 [ERROR] root:      - [auc] Plot 생성 실패: AUC plot not available for estimators with no predict_proba attribute.


  -> 모델 [RidgeClassifier]의 Plot 생성...


2025-08-28 23:36:55,415 [ERROR] root:      - [calibration] Plot 생성 실패: Calibration plot not available for estimators with no predict_proba attribute.


  -> 모델 [LogisticRegression]의 Plot 생성...
  -> 모델 [ExtraTreesClassifier]의 Plot 생성...
  -> 모델 [RandomForestClassifier]의 Plot 생성...
  -> 모델 [LGBMClassifier]의 Plot 생성...
  -> 모델 [GradientBoostingClassifier]의 Plot 생성...
  ✅ [5x] 데이터세트의 Plot 저장이 완료되었습니다.

--- 데이터세트 [10x]에 대한 PyCaret 세션 활성화 ---
--- [10x] 데이터세트 Plot 저장 시작 ---
  -> 모델 [AdaBoostClassifier]의 Plot 생성...


...

  -> 모델 [RidgeClassifier]의 Plot 생성...


2025-08-28 23:59:39,528 [WARNING] root:     - 기본 파일 'Prediction Error.png'을 찾을 수 없습니다.
2025-08-28 23:59:39,528 [ERROR] root:      - [calibration] Plot 생성 실패: Calibration plot not available for estimators with no predict_proba attribute.


  -> 모델 [LogisticRegression]의 Plot 생성...


2025-08-28 23:59:43,496 [WARNING] root:     - 기본 파일 'AUC.png'을 찾을 수 없습니다.
2025-08-28 23:59:47,131 [WARNING] root:     - 기본 파일 'Confusion Matrix.png'을 찾을 수 없습니다.
2025-08-29 00:00:18,101 [WARNING] root:     - 기본 파일 'Learning Curve.png'을 찾을 수 없습니다.


  -> 모델 [ExtraTreesClassifier]의 Plot 생성...


In [ ]:
## 모델 평가 (evaluation)
import pandas as pd
from pycaret.classification import setup, evaluate_model

# 전체 데이터세트 목록
data_suffixes = ['5x', '10x', '20x']

# all_tuned_models 딕셔너리를 순회
for suffix in data_suffixes:
    # 해당 suffix에 대한 결과가 있는지 확인
    if suffix in all_tuned_models:
        
        # [추가] 평가 전에 해당 데이터로 setup을 다시 실행하여 세션을 활성화
        print(f"\n\n=================================================")
        print(f"--- 데이터세트 [{suffix}]에 대한 PyCaret 세션 활성화 ---")
        x_train = globals()[f'x_train_{suffix}']
        y_train = globals()[f'y_train_{suffix}']
        df_train = pd.concat([x_train, y_train], axis=1)
        
        # 튜닝 때 사용했던 설정과 동일하게 setup을 실행
        setup(
            data=df_train,
            target='potency',
            session_id=42,
            train_size=0.9,
            fold=10,
            normalize=True,
            fix_imbalance=True,
            remove_outliers=True,
            n_jobs=-1,
            verbose=False
        )
        
        print(f"--- 데이터세트 [{suffix}] 모델 평가 시작 ---")
        
        # 튜닝된 모델들을 하나씩 꺼내어 평가
        for model_name, model_object in all_tuned_models[suffix].items():
            print(f"\n>>> 모델 [{model_name}]의 평가 대시보드를 생성합니다...")
            
            # evaluate_model 실행
            evaluate_model(model_object)
    else:
        print(f"\n--- 데이터세트 [{suffix}]에 대한 튜닝 결과가 없습니다. 건너뜁니다. ---")

print("\n\n🎉 모든 모델 평가가 완료되었습니다!")

In [ ]:
## 모델 블렌딩

# 블렌딩된 모델을 저장할 딕셔너리
all_blended_models = {}

# 블렌딩 결과를 저장할 딕셔너리
all_blended_results = {}

# 블렌딩할 데이터세트 목록
data_suffixes = ['5x', '10x', '20x']

for suffix in data_suffixes:
    print(f"\n--- 데이터세트 [{suffix}] 모델 블렌딩 시작 ---")
    
    # 1. [핵심] 해당 데이터세트의 튜닝된 모델들을 F1 점수 기준으로 정렬
    summary_df = all_summary_dfs[suffix]
    mean_scores = summary_df[summary_df['Type'] == 'Mean'].copy()
    top_models_df = mean_scores.sort_values(by='F1', ascending=False)
    
    # 2. 상위 4개 모델의 이름(클래스명)을 리스트로 추출
    top_3_model_names = top_models_df['Model'].head(4).tolist()
    
    # 3. 이름에 해당하는 실제 모델 객체를 가져오기
    models_to_blend = [all_tuned_models[suffix][name] for name in top_3_model_names]
    
    print(f" -> 블렌딩 대상 모델: {top_3_model_names}")
    
    # 4. 모델 블렌딩 실행
    # method='soft'는 각 모델의 예측 '확률'을 평균내어 더 부드러운 결정을 내림
    blended_model = blend_models(
        estimator_list=models_to_blend, 
        optimize='F1',
        method='soft', 
        verbose=False
    )
    
    # 5. 블렌딩 결과(성능 점수) 저장
    blended_results_df = pull().copy()
    
    # 6. 각 딕셔너리에 결과 저장
    all_blended_models[suffix] = blended_model
    all_blended_results[suffix] = blended_results_df
    
    print(" -> 블렌딩 후 성능:")
    display(blended_results_df)

print("\n🎉 모든 데이터세트의 모델 블렌딩이 완료되었습니다!")

In [ ]:
## 전체 데이터세트 대상, 블렌딩 모델 최종 성능 평가
import pandas as pd

# 1. 전체 데이터세트 목록 정의
data_suffixes = ['5x', '10x', '20x']

# 최종 예측 결과와 성능 지표를 저장할 딕셔너리
all_final_predictions = {}
all_final_metrics = {}

print(f"\n=============== 전체 데이터세트 최종 성능 평가 시작 ===============")

# 2. for 반복문을 사용하여 모든 데이터세트를 순회
for suffix in data_suffixes:
    print(f"\n--- [{suffix}] 데이터세트 평가 중 ---")
    
    # 해당 suffix에 대한 블렌딩 모델이 있는지 확인
    if suffix in all_blended_models:
        # 3. 해당 데이터세트의 블렌딩 모델 및 테스트 데이터 준비
        blended_model = all_blended_models[suffix]
        x_test = globals()[f'x_test_{suffix}']
        y_test = globals()[f'y_test_{suffix}']
        test_df = pd.concat([x_test, y_test], axis=1)

        # 4. predict_model을 사용하여 최종 성능 평가
        final_predictions = predict_model(blended_model, data=test_df)
        
        # 5. 결과 저장
        all_final_predictions[suffix] = final_predictions
        all_final_metrics[suffix] = pull() # predict_model 직후의 성능표 저장
        
    else:
        print(f" -> [{suffix}]에 대한 블렌딩 모델이 없어 건너뜁니다.")

print("\n\n🎉 모든 데이터세트에 대한 최종 평가가 완료되었습니다!")

# 6. 전체 결과 요약 출력
print("\n\n=============== 최종 성능 요약 ===============")
for suffix, metrics_df in all_final_metrics.items():
    print(f"\n--- [{suffix}] Blended Model Final Score ---")
    display(metrics_df)

임시코드

In [ ]:
## 모델 블랜딩

# 1. 블렌딩할 모델들이 있는 데이터세트 선택
target_suffix = '1x'

# 2. 해당 데이터세트의 튜닝된 모델 객체들을 리스트로 만들기
# all_tuned_models 딕셔너리에서 모델 객체(value)들만 가져옵니다.
models_to_blend = list(all_tuned_models[target_suffix].values())

print(f"[{target_suffix}] 데이터세트의 다음 모델들을 블렌딩합니다:")
for model in models_to_blend:
    print(f" - {str(model).split('(')[0]}")

# 3. 모델 블렌딩 실행
# fold=5는 교차 검증을 5번 수행한다는 의미입니다.
blended_model = blend_models(estimator_list=models_to_blend, 
                             fold=5,
                             optimize='F1')

# 4. 블렌딩된 모델의 성능 확인
print("\n--- 블렌딩된 모델의 성능 ---")
blended_results = pull()
display(blended_results)

In [ ]:
## 블랜딩 모델 성능평가

# 1. 블렌딩 모델을 만들 때 사용했던 데이터세트 지정
# 이전 단계에서 '1x'를 사용했으므로 동일하게 설정합니다.
target_suffix = '1x' 

print(f"\n=============== 블렌딩 모델 최종 성능 평가 시작 ({target_suffix}) ===============")

# 2. 해당 데이터세트의 테스트 데이터 준비
x_test = globals()[f'x_test_{target_suffix}']
y_test = globals()[f'y_test_{target_suffix}']
test_df = pd.concat([x_test, y_test], axis=1)

# 3. predict_model을 사용하여 블렌딩 모델의 최종 성능 평가
blended_model_predictions = predict_model(blended_model, data=test_df)

print("\n🎉 블렌딩 모델의 최종 평가가 완료되었습니다!")